Step2: Embedding-based Model (public score: 1.03647)

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

llm_classification_finetuning_path = kagglehub.competition_download('llm-classification-finetuning')
shinomoriaoshi_sentencetransformersallminilml6v2_path = kagglehub.dataset_download('shinomoriaoshi/sentencetransformersallminilml6v2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!pip install sentence-transformers lightgbm pandas numpy
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentencetransformersallminilml6v2/config.json
/kaggle/input/sentencetransformersallminilml6v2/tokenizer.json
/kaggle/input/sentencetransformersallminilml6v2/tokenizer_config.json
/kaggle/input/sentencetransformersallminilml6v2/pytorch_model.bin
/kaggle/input/sentencetransformersallminilml6v2/special_tokens_map.json
/kaggle/input/sentencetransformersallminilml6v2/vocab.txt
/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
^C


In [ ]:
# --- 1. 데이터 로드 ---
import warnings


warnings.filterwarnings('ignore')


# 캐글 환경의 데이터 경로입니다.
# 본인의 캐글 노트북 환경에 맞게 경로를 확인하세요.
DATA_PATH = "/kaggle/input/llm-classification-finetuning/"

try:
    train_df = pd.read_csv(f"{DATA_PATH}train.csv")
    test_df = pd.read_csv(f"{DATA_PATH}test.csv")
    sample_submission_df = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

    print(f"학습 데이터 크기: {train_df.shape}")
    print(f"테스트 데이터 크기: {test_df.shape}")

except FileNotFoundError:
    print(f"오류: 캐글 데이터셋 경로({DATA_PATH})를 찾을 수 없습니다.")
    print("로컬 환경이거나 경로가 다른 경우, DATA_PATH 변수를 수정해주세요.")


학습 데이터 크기: (57477, 9)
테스트 데이터 크기: (3, 4)


In [ ]:
# --- 2. 타겟 변수(y_train) 생성 ---
import pandas as pd
import numpy as np
# 'winner_model_a', 'winner_model_b', 'winner_model_tie' 3개 컬럼을
# 0, 1, 2 (a, b, tie) 형태의 단일 숫자 레이블로 변환합니다.
target_cols = ['winner_model_a', 'winner_model_b', 'winner_tie']
# np.argmax는 [1, 0, 0] -> 0, [0, 1, 0] -> 1, [0, 0, 1] -> 2로 바꿔줍니다.
y_train = np.argmax(train_df[target_cols].values, axis=1)

print("\n타겟 변수(y_train) 생성 완료.")
print(f"y_train 분포:\n{pd.Series(y_train).value_counts(normalize=True)}")


타겟 변수(y_train) 생성 완료.
y_train 분포:
0    0.349079
1    0.341911
2    0.309011
Name: proportion, dtype: float64


In [ ]:
!ls -al /kaggle/input/sentencetransformersallminilml6v2

total 89700
drwxr-xr-x 2 nobody nogroup        0 Oct 29 10:54 .
drwxr-xr-x 4 root   root        4096 Nov  5 04:04 ..
-rw-r--r-- 1 nobody nogroup      678 Oct 29 10:54 config.json
-rw-r--r-- 1 nobody nogroup 90890157 Oct 29 10:54 pytorch_model.bin
-rw-r--r-- 1 nobody nogroup      125 Oct 29 10:54 special_tokens_map.json
-rw-r--r-- 1 nobody nogroup      570 Oct 29 10:54 tokenizer_config.json
-rw-r--r-- 1 nobody nogroup   711661 Oct 29 10:54 tokenizer.json
-rw-r--r-- 1 nobody nogroup   231508 Oct 29 10:54 vocab.txt


In [ ]:
# --- 3. 임베딩 모델 로드 ---
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np


MODEL_PATH = "/kaggle/input/sentencetransformersallminilml6v2"
# 'all-MiniLM-L6-v2'는 작고 빠르며 준수한 성능을 내는 모델입니다.
print(f"\n임베딩 모델 로드 중... (로컬 경로: {MODEL_PATH})")

try:
    embedding_model = SentenceTransformer(
        MODEL_PATH,
        device='cuda',
        # 1. local_files_only=True : 인터넷(Hugging Face Hub) 조회를 강제로 차단합니다.
        local_files_only=True,
        # 2. tokenizer_kwargs : 'additional_chat_templates' 404 오류를 우회합니다.
        tokenizer_kwargs={"chat_template": None}
    )
    print("모델 로드 완료.")

except OSError:
    print(f"오류: {MODEL_PATH} 경로에 모델이 없습니다.")
    print("오른쪽 'Add Data'에서 'sentence-transformers/all-minilm-l6-v2' 데이터셋을 추가했는지 확인하세요.")
    # 오류 발생 시에도 스크립트가 멈추지 않도록 예외 처리
    raise Exception("모델 로드 실패. 캐글 데이터셋을 추가하세요.")

print("모델 로드 완료.")

# --- 4. 텍스트 임베딩 생성 함수 ---
def get_embeddings(df, model):
    print(f"총 {len(df)}개의 데이터 임베딩 생성 중...")

    # 각 텍스트 컬럼을 리스트로 변환
    prompts = df['prompt'].tolist()
    responses_a = df['response_a'].tolist()
    responses_b = df['response_b'].tolist()

    # 모델로 임베딩
    print("Prompt 임베딩 중...")
    prompt_emb = model.encode(prompts, show_progress_bar=True, device='cuda')
    print("Response A 임베딩 중...")
    resp_a_emb = model.encode(responses_a, show_progress_bar=True, device='cuda')
    print("Response B 임베딩 중...")
    resp_b_emb = model.encode(responses_b, show_progress_bar=True, device='cuda')

    return prompt_emb, resp_a_emb, resp_b_emb

# --- 5. 피처(X) 생성 함수 ---
# A vs B 비교 문제에서는 각 임베딩을 합치는(concatenate) 것 외에도
# A와 B의 차이(difference), 곱(element-wise product)을 피처로 넣어주면 성능이 향상됩니다.
def create_features(prompt_emb, resp_a_emb, resp_b_emb):
    diff_emb = resp_a_emb - resp_b_emb
    prod_emb = resp_a_emb * resp_b_emb

    # (prompt, resp_a, resp_b, a-b, a*b) 5개 임베딩을 가로로 연결
    features = np.concatenate([
        prompt_emb,
        resp_a_emb,
        resp_b_emb,
        diff_emb,
        prod_emb
    ], axis=1)

    return features

# 학습 데이터 피처 생성
train_prompt_emb, train_resp_a_emb, train_resp_b_emb = get_embeddings(train_df, embedding_model)
X_train_features = create_features(train_prompt_emb, train_resp_a_emb, train_resp_b_emb)

print(f"학습 피처 행렬 크기: {X_train_features.shape}") # (n_samples, 384 * 5)

# 테스트 데이터 피처 생성
test_prompt_emb, test_resp_a_emb, test_resp_b_emb = get_embeddings(test_df, embedding_model)
X_test_features = create_features(test_prompt_emb, test_resp_a_emb, test_resp_b_emb)

print(f"테스트 피처 행렬 크기: {X_test_features.shape}")


임베딩 모델 로드 중... (로컬 경로: /kaggle/input/sentencetransformersallminilml6v2)
모델 로드 완료.
모델 로드 완료.
총 57477개의 데이터 임베딩 생성 중...
Prompt 임베딩 중...


Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Response A 임베딩 중...


Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Response B 임베딩 중...


...

Response A 임베딩 중...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Response B 임베딩 중...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

테스트 피처 행렬 크기: (3, 1920)


In [ ]:
from sentence_transformers import SentenceTransformer
import lightgbm as lgb
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
# --- 6. 분류 모델 학습 (LightGBM) ---
print("\nLightGBM 모델 학습 시작...")

# K-Fold 교차 검증을 사용하는 것이 훨씬 안정적이고 좋지만,
# 우선 전체 데이터로 학습합니다.
lgb_classifier = lgb.LGBMClassifier(
    objective='multiclass',  # 다중 분류 (a, b, tie)
    metric='multi_logloss',  # 대회의 평가 지표
    num_class=3,             # 클래스 개수 (0, 1, 2)
    random_state=42,
    n_estimators=300,        # 트리의 개수 (튜닝 필요)
    learning_rate=0.03,      # 학습률 (튜닝 필요)
    num_leaves=31,           # 리프 노드의 수 (튜닝 필요)
    device='gpu'
    # n_jobs=-1                # 모든 CPU 코어 사용
)

lgb_classifier.fit(X_train_features, y_train)

print("모델 학습 완료.")



LightGBM 모델 학습 시작...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 489090
[LightGBM] [Info] Number of data points in the train set: 57477, number of used features: 1918
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1918 dense feature groups (105.24 MB) transferred to GPU in 0.099271 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -1.052458
[LightGBM] [Info] Start training from score -1.073206
[LightGBM] [Info] Start training from score -1.174380
모델 학습 완료.


In [ ]:
# --- 7. 예측 ---
print("테스트 데이터로 예측 수행 중...")
# Log Loss 평가를 위해 각 클래스(a, b, tie)의 확률을 예측합니다.
# test_probabilities는 (n_test_samples, 3) 크기의 배열이 됩니다.
test_probabilities = lgb_classifier.predict_proba(X_test_features)

print("예측 완료.")

# --- 8. 제출 파일 생성 ---
print("제출 파일(submission.csv) 생성 중...")

# sample_submission.csv는 id만 사용하고, 3개의 확률 컬럼을 추가합니다.
submission_df = pd.DataFrame({'id': test_df['id']})
submission_df['winner_model_a'] = test_probabilities[:, 0]  # 클래스 0 (a)의 확률
submission_df['winner_model_b'] = test_probabilities[:, 1]  # 클래스 1 (b)의 확률
submission_df['winner_model_tie'] = test_probabilities[:, 2]  # 클래스 2 (tie)의 확률

submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' 파일이 성공적으로 생성되었습니다.")
print("캐글에 이 파일을 제출하여 Step 2 점수를 확인하세요!")

print("\n--- 스크립트 종료 ---")

테스트 데이터로 예측 수행 중...
예측 완료.
제출 파일(submission.csv) 생성 중...

'submission.csv' 파일이 성공적으로 생성되었습니다.
캐글에 이 파일을 제출하여 Step 2 점수를 확인하세요!

--- 스크립트 종료 ---
